### ANN/MLP 모델 구현
- 구성 : 전결합층(FC Layer)을 여러개로 구성
- 학습 : 회귀, 분류, 다중 분류

[1] 모듈 로딩 및 데이터 준비

In [19]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [20]:
### 데이터 준비
DATA_FILE='./iris.csv'

### 데이터 로딩
### Header 파라미터 : 데이터 파일의 첫번째 줄이 컬럼명 여부 체크
### usecols 파라미터 : 로딩하고 싶은 컬럼 지정
iris_DF = pd.read_csv(DATA_FILE, header=None, usecols=[0,1,2,3])
iris_DF.head(5)

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


[2] 모델 클래스 설계 및 정의
- 전결합층 클래스 => DenseModel
- 컬렉션 클래스 => SeqenceModel

In [21]:
### 전결합층 클래스 

class DenseModel(tf.Module):
    def __init__(self, input_nums, out_nums):
        super().__init__()
        self.W = tf.Variable(tf.random.normal((input_nums, out_nums)))
        self.b = tf.Variable(tf.zeros((out_nums)))
    
    def __call__(self, input):
        wbsum = tf.matmul(input, self.W) + self.b

        return wbsum

In [30]:
### -------------------------------------------------------
### 컬렉션 클래스
### 클래스 목적 : 모델의 층 구성 클래스
### 클래스 이름 : SequenceModel
### 클래스 속성 : in_layer, d_layer, out_layer
### 클래스 기능 : 층별 학습 진행 후 결과 전달 즉, 전방향 학습 기능
### -------------------------------------------------------

class SequenceModel(tf.Module):
    #인스턴스 생성 메서드
    def __init__(self, in_nums, out_nums, kind='reg'):
        super().__init__()
        self.kind = kind
        
        #입력층 : 피쳐개수, 퍼셉트론개수
        self.in_layer = DenseModel(in_nums, in_nums*2)

        #은닉층 : 피쳐개수, 퍼셉트론개수
        self.hd_layer = DenseModel(in_nums*2, in_nums)
        
        #출력층 : 피쳐개수, 타켓개수
        self.out_layer = DenseModel(in_nums,out_nums)

    # 순방향/전방향 학습 메서드
    def __call__(self, data):
        #입력층 학습
        wbsum = self.in_layer(data)
        output = tf.nn.relu(wbsum)

        #은닉층 학습
        wbsum = self.hd_layer(output)
        output = tf.nn.relu(wbsum)

        #출력층 학습
        if self.kind=='reg':
            return self.out_layer(output)
        elif self.kind == 'binary':
            return tf.nn.sigmoid(self.out_layer(output))
        elif self.kind == 'multi':
            return tf.nn.softmax(self.out_layer(output))
        else:
            return 'unknown'

[3] 학습 준비
- 데이터셋 -> 피쳐와 타켓 분리
- 인스턴스/객체 -> 모델 인스턴스, 최적화 인스턴스, 손실함수 인스턴스
- 학습 관련 함수들 -> 학습 함수, 검증함수, 예측함수

[3-1] 피쳐와 타켓 데이터셋 분리

In [23]:
### 텐서화 - 변경불가 데이터 즉, tf.constant()
# DataFrame -> Numpy ndarray
feature_NP = iris_DF[ iris_DF.columns[:-1] ].values
target_NP = iris_DF[ iris_DF.columns[-1:] ].values

print(f'feature_NP shape -> {feature_NP.shape}, target_NP shape -> {target_NP.shape}')

feature_NP shape -> (150, 3), target_NP shape -> (150, 1)


In [24]:
#Numpy ndarray --> Tensor
feature_TS = tf.constant(feature_NP, dtype=tf.float32)
target_TS = tf.constant(target_NP, dtype=tf.float32)

print(f'feature_TS shape -> {feature_NP.shape}, target_TS shape -> {target_NP.shape}')

feature_TS shape -> (150, 3), target_TS shape -> (150, 1)


[3-2] 모델 인스턴스 생성

In [25]:
# 입력 피쳐 수와 츨력 층 퍼셉트론 수 즉, 타겟 수
IN_NUMS, OUT_NUMS = feature_TS.shape[1], target_TS.shape[1]

print(f'IN_NUMS -> {IN_NUMS}, OUT_NUMS -> {OUT_NUMS}')

IN_NUMS -> 3, OUT_NUMS -> 1


In [32]:
# 모델 인스턴스 생성
iris_Model = SequenceModel(IN_NUMS, OUT_NUMS)

In [33]:
# 테스트
y_pre = iris_Model(feature_TS)

print(f'{y_pre}')

[[2.4062881]
 [2.3229363]
 [2.2231696]
 [2.3580801]
 [2.3970733]
 [2.742866 ]
 [2.3045666]
 [2.4598613]
 [2.2212148]
 [2.4135482]
 [2.5616426]
 [2.5042198]
 [2.3044472]
 [1.967989 ]
 [2.4194126]
 [2.6820326]
 [2.4175174]
 [2.4062881]
 [2.7890594]
 [2.515449 ]
 [2.696493 ]
 [2.5061743]
 [1.9977673]
 [2.6317503]
 [2.7482307]
 [2.5041003]
 [2.5411983]
 [2.5061145]
 [2.415503 ]
 [2.467181 ]
 [2.4763958]
 [2.533819 ]
 [2.5617619]
 [2.5451677]
 [2.4135482]
 [2.1973007]
 [2.3989086]
 [2.4135482]
 [2.1491525]
 [2.4783509]
 [2.3064616]
 [2.1027198]
 [2.1677015]
 [2.550473 ]
 [2.8407972]
 [2.3044472]
 [2.596786 ]
 [2.2860174]
 [2.543153 ]
 [2.3692496]
 [5.4138856]
 [5.140275 ]
 [5.5487967]
 [4.483715 ]
 [5.2030034]
 [4.973752 ]
 [5.293735 ]
 [3.812693 ]
 [5.2307677]
 [4.384008 ]
 [3.956759 ]
 [4.785268 ]
 [4.566887 ]
 [5.219658 ]
 [4.2325025]
 [5.1051326]
 [4.973811 ]
 [4.657618 ]
 [5.0105515]
 [4.439416 ]
 [5.2918396]
 [4.641024 ]
 [5.3822126]
 [5.210383 ]
 [4.9497786]
 [5.0773683]
 [5.4211464]